In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt


In [1]:
!pip install tensorflow
!pip install numpy
!pip install matplotlib
!pip install pillow

  Using cached protobuf-5.29.3-cp310-abi3-win_amd64.whl.metadata (592 bytes)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached setuptools-75.8.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
  Using cached numpy-2.0.2-cp312-cp312-win_amd64.whl.metadata (59 kB)
  Using cached wheel-0.45.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
     ---------------------------------------- 0.0/48.6 kB ? eta -:--:--
     ---------------------------------------- 48.6/48.6 kB 1.2 MB/s eta 0:00:00
  Using cached charset_normalizer-3.4.1-cp312-cp312-win_amd64.whl.metadata (36 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2024.12.14-py3-none-any.whl.metadata (2.3 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0

ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'C:\\Users\\theve\\OneDrive\\Desktop\\Running_Projects\\Instagram_adult_contant_dection\\adultEnv\\Lib\\site-packages\\tensorflow\\include\\tensorflow\\compiler\\xla\\mlir_hlo\\_virtual_includes\\chlo_legalize_to_hlo_inc_gen\\chlo_legalize_to_hlo\\generated_chlo_legalize_to_hlo.inc'
HINT: This error might have occurred since this system does not have Windows Long Path support enabled. You can find information on how to enable this at https://pip.pypa.io/warnings/enable-long-paths


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/169.5 kB ? eta -:--:--
     ---- -------------------------------- 20.5/169.5 kB 682.7 kB/s eta 0:00:01
     ---- -------------------------------- 20.5/169.5 kB 682.7 kB/s eta 0:00:01
     ----------------- ------------------- 81.9/169.5 kB 657.6 kB/s eta 0:00:01
     --------------------- -------------- 102.4/169.5 kB 658.3 kB/s eta 0:00:01
     ---------------------------------- - 163.8/169.5 kB 821.4 kB/s eta 0:00:01
     ------------------------------------ 169.5/169.5 kB 784.5 kB/s eta 0:00:00
  Using cached pillow-11.1.0-cp312-cp312-win_amd64.whl.metadata (9.3 kB)
   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.0 MB 1.4 MB/s eta 0:00:06
   ---------------------------------------- 0.0/8.0 MB 495.5 kB/s eta 0:00:17
   ---------------------------------------- 0.1/8.0 MB 657.6 kB/s eta 0:00:13
   ---------------------------------------- 0.1/8.0 MB 585.1 kB/s eta 0:0


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
train_dir = 'data/trein'

batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary'  # Since we have two classes
)

class_names = train_dataset.class_names
print("Class names:", class_names)  # Should print ['1', '2']


In [ ]:
train_dir = 'data/trein'

batch_size = 32
img_height = 224
img_width = 224

train_dataset = image_dataset_from_directory(
    train_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode='binary'  # Since we have two classes
)

class_names = train_dataset.class_names
print("Class names:", class_names)  # Should print ['1', '2']


In [ ]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
])

normalization_layer = layers.Rescaling(1./255)
train_dataset = train_dataset.map(lambda x, y: (normalization_layer(x), y))


In [ ]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_height, img_width, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification output
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


In [ ]:
epochs = 10
history = model.fit(train_dataset, epochs=epochs)


In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']

plt.figure(figsize=(8, 4))
plt.plot(range(epochs), acc, label='Training Accuracy')
plt.plot(range(epochs), loss, label='Training Loss')
plt.legend()
plt.title('Training Performance')
plt.show()


In [ ]:
def predict_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_height, img_width))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) / 255.0
    
    prediction = model.predict(img_array)
    if prediction < 0.5:
        print("Good Picture")
    else:
        print("Nude Picture")

predict_image('data/trein/1/sample.jpg')  # Test with an image
